# Create a Question Answering Chat



This is a quick template for creating a question answering chat with ChatGPT and 🦜🔗 LangChain.



We load an example document and create an index using OpenAI text embeddings. Then, we can chat about the contents of this document.

In [ ]:
%setup langchain openai chromadb

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma

from langchain.text_splitter import CharacterTextSplitter

from langchain.llms import OpenAI

from langchain.chat_models import ChatOpenAI

from langchain.chains import ConversationalRetrievalChain

from langchain.document_loaders import TextLoader

from langchain.memory import ConversationBufferMemory

import urllib.request



# retrieve the state of the union speech

urllib.request.urlretrieve("https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt", "state_of_the_union.txt")



txts = [

    "state_of_the_union.txt"

];



documents = []



for txt in txts:

    loader = TextLoader(txt)

    docs = loader.load()

    documents.extend(docs)



text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(documents, embeddings)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")

llm = ChatOpenAI(temperature=0)

# if you want GPT-4: 

# llm = ChatOpenAI(temperature=0, model_name="gpt-4")



qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), memory=memory, get_chat_history=lambda inputs: inputs)